In [26]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
from nltk.stem.porter import PorterStemmer
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from nltk import bigrams
from nltk.tokenize import word_tokenize
import re
%run part1_data_collection.ipynb


# Load Spacy English Model
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Etudiant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
#Segmentation in sentences and tokens and vocabulary
def sentence_segmentation(spacy_text):
    list_of_sentences = []
    for s in spacy_text.sents:
        list_of_sentences.append(s)
    return list_of_sentences

def tokens_segmentation(spacy_text):
    list_of_tokens = []
    for tok in spacy_text:
        list_of_tokens.append(str(tok))
    return list_of_tokens


def lowercase_and_punctuation_remover(spacy_list):
    translator = str.maketrans('', '', string.punctuation) #with string.punctuation being the list of punctuation signs
    return [tok.translate(translator) for tok in [str(tok).lower() for tok in spacy_list]]


def remove_stopwords(spacy_list):
    #print(stopwords.words('english')[:100])
    stopwords_list = stopwords.words('english')
    spacy_list_clean = []
    for token in spacy_list:
        if token not in stopwords_list :
            spacy_list_clean.append(token)
    return spacy_list_clean

def remove_empty_spaces(token_list):
    # Strip each token in the list and remove it if it's empty after stripping
    return [token.strip() for token in token_list if token.strip()]

def stemming(tokens_list):
    # Create a stemmer instance
    porter = PorterStemmer()
    # Apply the stemmer and collect the stems
    porterlemmas = []
    for word in tokens_list:
        porterlemmas.append(porter.stem(word))
    return porterlemmas

def text_data_tok_sent(df):
    #sentences_journalists = []
    #sentences_sculptors = []
    #tokens_journalists = []
    #tokens_sculptors =[]
    #for index, row in df.iterrows():
    #    if(row['category'] == 'Journalists'):
    #        sentences_journalists.extend(sentence_segmentation(nlp(row['text'])))
    #    elif(row['category'] == 'Sculptors'):
    #        sentences_sculptors.extend(sentence_segmentation(nlp(row['text'])))
    
    df['sentences'] = df['text'].apply(lambda x: lowercase_and_punctuation_remover(sentence_segmentation(nlp(x))))
    #sentences_journalists_clean = lowercase_and_punctuation_remover(sentences_journalists)
    #sentences_sculptors_clean = lowercase_and_punctuation_remover(sentences_sculptors)
                                         
    df['tokens'] = df['sentences'].apply(lambda x: stemming(remove_empty_spaces(remove_stopwords(tokens_segmentation(nlp(' '.join(x)))))))
    
    #tokens_journalists = stemming(remove_empty_spaces(remove_stopwords(tokens_segmentation(nlp(' '.join(sentences_journalists_clean))))))
    #tokens_sculptors = stemming(remove_empty_spaces(remove_stopwords(tokens_segmentation(nlp(' '.join(sentences_sculptors_clean))))))
                                         
    return df #(sentences_journalists_clean, sentences_sculptors_clean, tokens_journalists, tokens_sculptors)

def frequency_and_wordcloud(df):
    categories = df['category'].unique()
    results = {}
    
    for category in categories:
        print(f"50 most frequent words and wordcloud for {category}:")
        tokens_list = [item for sublist in df[df['category'] == category]['tokens'] for item in sublist]
        frequency_dist=nltk.FreqDist(tokens_list)
        frequency_dist.plot(50)
        #Create and generate a word cloud image:
        wordcloud = WordCloud().generate(" ".join(tokens_list))
        # Display the generated image:
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
    
def data_analysis(df): 
    df = text_data_tok_sent(df)
    frequency_and_wordcloud(df)
    return df
    


In [28]:
def sentences(df):
    
    # Sentence segmentation and count
    df['sentence_count'] = df['sentences'].apply(lambda x: len(x))

    # min, max, and mean sentence counts by category
    stats_df = df.groupby('category')['sentence_count'].agg(['min', 'max', 'mean'])

    # print statistics
    print(stats_df)
    
    # visualization
    categories = df['category'].unique()
    fig, ax = plt.subplots(1, len(categories)+1, figsize=(12, 6))
    counter = 0 #to keep track of the number of a categorie for subplot index
    for category in categories:
        print(df[df["category"] == category])
        sns.histplot(df[df["category"] == category], x = 'sentence_count', ax=ax[counter])

        # set titles for each subplot for clarity
        ax[counter].set_title(f'Sentences length frequency for {category}')
        ax[counter].axvline(stats_df.loc[category, 'mean'], color='r', linestyle='--', linewidth=2, label=f'Mean: {stats_df.loc[category, "mean"]:.2f}')
        counter += 1
    
    sns.boxplot(data=df, x='category', y='sentence_count', ax=ax[counter])
    ax[counter].set_title('Boxplot')
    plt.show()

In [29]:
#bigrams

def generate_bigrams(sentences_list):
        all_bigrams = []
        for sent in sentences_list:
            # bigrams from the list of sentences
            tokens = word_tokenize(sent)
            all_bigrams.append(list(bigrams(tokens))) #list of list of bigrams (one per sentence)
        return all_bigrams


def get_bigrams_list(df):
    df['bigrams'] = df['sentences'].apply(generate_bigrams)
    return df
    
def total_bigram_occ(df):
    pers_bigrams = {}
    sent_bigrams = {}
    all_bigrams = {}
    
    categories = df['category'].unique()
    for category in categories:
        pers_bigrams[category] = [x for x in df[df["category"] == category]['bigrams']]
        sent_bigrams[category] = sum(pers_bigrams[category], [])
        all_bigrams[category] = sum(sent_bigrams[category], [])
        print(f"For the category {category}:")
        print(f"  there are {len(all_bigrams[category])} bigrams in total and {len(set(all_bigrams[category]))} unique bigrams")
        print(f"  the minimum number of bigrams in a sentence is {min([len(s) for s in sent_bigrams[category]])}")
        print(f"  the maximum number of bigrams in a sentence is {max([len(s) for s in sent_bigrams[category]])}")
        print(f"  the average number of bigrams in a sentence is {sum([len(s) for s in sent_bigrams[category]])/len(sent_bigrams[category])}")


Graphs

In [30]:

def retrieve_json(path):
    json_string = open(path)
    
    graph = json.load(json_string)
    return graph

def extract_after_last_slash(text): 
    # for facts, regex to find the part after the last slash (to keep only the fact and not the url)
    match = re.search(r'[^/]+$', text)
    if match:
        return match.group(0)
    return None

def most_freq_properties_and_cloud(json_str):
    properties_per_category = {}
    properties_per_category_clean = {}

    for category, items in json_str.items():
        properties_per_category[category] = []
        properties_per_category_clean[category] = []

        for item in items:
            # extract all 'p' values from the bindings
            bindings = item['results']['bindings']
            for binding in bindings:
                property_value = binding['p']['value']
                #append property
                properties_per_category[category].append(property_value)
                properties_per_category_clean[category] = [extract_after_last_slash(prop) for prop in properties_per_category[category]]

        print(f"50 most frequent properties and wordcloud of properties for {category}:")
        frequency_dist_dist=nltk.FreqDist(properties_per_category_clean[category])
        frequency_dist_dist.plot(50)
        #Create and generate a word cloud image:
        wordcloud_prop = WordCloud().generate(" ".join(properties_per_category_clean[category]))
        # Display the generated image:
        plt.imshow(wordcloud_prop, interpolation='bilinear')
        plt.axis("off")
        plt.show()
        
    #print(properties_per_category['Journalists'])

In [31]:
df = createBiographyDataFrames()
df
#df = data_analysis(df)
#sentences(df)
#df = get_bigrams_list(df)
#total_bigram_occ(df)
#most_freq_properties_and_cloud(retrieve_json('knowledge_graph.json'))

Unexpected err=IndexError('list index out of range'), type(err)=<class 'IndexError'>, could not process Margaret Trowell


c:\Users\Etudiant\M1_TAL\m1-datascience-project\.venv\Lib\site-packages\SPARQLWrapper\Wrapper.py:794: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'DESCRIBE' SPARQL query form
  warnings.warn(


Sculptors Pavlos Prosalentis processed
Sculptors George Lilanga processed
Sculptors Eric Gill works at the Midland Hotel, Morecambe processed
Sculptors New British Sculpture processed
Sculptors New Sculpture processed
Sculptors Orestes Acquarone processed
Sculptors Georges Adéagbo processed
Sculptors Kudjoe Affutu processed
Sculptors Julio Aguilera processed
Sculptors Adebisi Akanji processed
Sculptors Kwame Akoto-Bamfo processed
Sculptors Sunday Jack Akpan processed
Sculptors Enrique Alférez processed
Sculptors José Simões de Almeida processed
Sculptors Julio Alpuy processed
Sculptors Maximiano Alves processed
Sculptors Eric Adjetey Anang processed
Sculptors El Anatsui processed
Sculptors Mark Antokolsky processed
Sculptors Massoud Arabshahi processed
Sculptors Carmelo Arden Quin processed
Sculptors Pablo Atchugarry processed
Sculptors Kevin Atherton processed
Sculptors Armando de Basto processed
Sculptors Alberto Bautista Gómez processed
Sculptors José Belloni processed
Sculptors Ste

,text,category,person
0,Pavlos Prosalentis (Greek: Παύλος Προσαλέντης;...,Sculptors,PavlosProsalentis
1,George Lilanga (1934 – 27 June 2005) was a Tan...,Sculptors,GeorgeLilanga
2,"Eric Gill works at the Midland Hotel, Morecamb...",Sculptors,"EricGillworksattheMidlandHotel,Morecambe"
3,New British Sculpture is the name given to the...,Sculptors,NewBritishSculpture
4,New Sculpture was a movement in late 19th-cent...,Sculptors,NewSculpture
...,...,...,...
255,Charles Graham Halpine (Halpin) (pseud. Miles ...,Journalists,CharlesG.Halpine
256,Mohammed Hanif (born November 1964) is a Briti...,Journalists,MohammedHanif
257,"Oscar Haza (born December 12, 1951) is a journ...",Journalists,ÓscarHaza
258,Mohamed Hassanein Heikal (Arabic: محمد حسنين ه...,Journalists,MohamedHassaneinHeikal
